# 07 - Generate statistics CSVs
Creating a separate notebook to calculate the statistics all in one pass so that I can complete a representative set of CSVs for each instrument class.

### Statistics for QARTOD tests in production

In [1]:
# Import libraries available from main conda channels or conda-forge
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import re
import warnings
warnings.filterwarnings("ignore")

# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

# Import qartod_testing project functions
from qartod_testing.data_processing import ooinet_gold_copy_request, get_test_parameters, parse_qartod_executed, qartod_summary_expanded

# Import OOI library functions
from ooi_data_explorations.common import merge_frames
from ooinet.M2M import get_deployments, get_annotations

In [2]:
# Setup parameters needed to request data
refdes = "GS01SUMO-RID16-03-CTDBPF000"
method = "recovered_host"
stream = "ctdbp_cdef_dcl_instrument_recovered"

# Site, node, and sensor info from deconstructed reference designator
# [site, node, sensor] = refdes.split('-', 2)

In [3]:
# Routine in data_processing module from this project to download the gold copy THREDDs datasets
# Variable 'files' contains list of catalog URLs for downloaded datasets 
files = ooinet_gold_copy_request(refdes, method, stream)

In [4]:
# Load expected results data from external data folder
folder_path = os.path.join(os.path.abspath('../data/external'), method, stream, refdes)
expected_files = glob.glob(folder_path+'/*.nc')
expected_files.sort() # sorts local test files in alphanumeric order

In [5]:
expected_files

['/home/jovyan/code/qartod_testing/data/external/recovered_host/ctdbp_cdef_dcl_instrument_recovered/GS01SUMO-RID16-03-CTDBPF000/deployment0001_GS01SUMO-RID16-03-CTDBPF000-recovered_host-ctdbp_cdef_dcl_instrument_recovered_20150218T211507.035000-20150611T000008.454000.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_host/ctdbp_cdef_dcl_instrument_recovered/GS01SUMO-RID16-03-CTDBPF000/deployment0002_GS01SUMO-RID16-03-CTDBPF000-recovered_host-ctdbp_cdef_dcl_instrument_recovered_20151214T202006.149000-20160921T235958.584000.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_host/ctdbp_cdef_dcl_instrument_recovered/GS01SUMO-RID16-03-CTDBPF000/deployment0002_GS01SUMO-RID16-03-CTDBPF000-recovered_host-ctdbp_cdef_dcl_instrument_recovered_20160922T000010.584000-20161212T075758.981000.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_host/ctdbp_cdef_dcl_instrument_recovered/GS01SUMO-RID16-03-CTDBPF000/deployment0003_GS01SUMO-RID16-03-CTDBPF000-recovered_

In [6]:
get_annotations(refdes)

,@class,id,subsite,node,sensor,method,stream,beginDT,endDT,annotation,exclusionFlag,source,qcFlag,parameters
0,.AnnotationRecord,834,GS01SUMO,None,None,None,None,1512777600000,1544330520000,Deployment 3: *UPDATED 2020-04-28: This moorin...,False,lgarzio@marine.rutgers.edu,0,[]
1,.AnnotationRecord,2131,GS01SUMO,RID16,03-CTDBPF000,recovered_inst,None,1480036260000,1544330520000,Deployment 3: There is no recovered instrument...,False,cdobson@whoi.edu,9,[]
2,.AnnotationRecord,1577,GS01SUMO,RID16,None,None,None,1549216800000,1549886400000,UPDATED 2020-06-26: This data gap will also be...,False,cdobson@whoi.edu,0,[]


#### Gross range test statistics

In [7]:
def collect_statistics(file_paths, test_name):
    """
    Calls other functions to calculate statistics from a set of files and a name of a QARTOD test. The statistics are organized in a DataFrame.
    
    Parameters:
    -----------
        file_paths: list of paths to each file that will have statistics calculated. File names must include "deployment00##".
        test_name: string of QARTOD test name, i.e. "gross_range", "climatology".
        
    Returns:
    --------
        statistics: Pandas DataFrame containing statistics on each parameter with a QARTOD test in order of deployment number, then statistics of the full record.
        
    Version 16 Aug 2023, Kylene M Cooley    
    """
    
    # Initialize empty dictionary for statistics
    statistics = {}

    for m, _ in enumerate(file_paths):
        file = file_paths[m]

        # get deployment from current file, then open local test and expected test datasets
        deployment = re.findall('deployment00[0-2][0-9]', file)[0][-2:]
        file_ds = xr.open_dataset(file)

        # Get parameters that have QARTOD executed from expected test dataset
        test_parameters = get_test_parameters(file_ds)
        parameters = list(test_parameters.keys())

        # Separate QARTOD test flags in expected test dataset by QARTOD test name
        file_ds = parse_qartod_executed(file_ds, parameters)

        # Update summary statistics dictionary for each deployment, then for all deployments
        print("Evaluating statistics on QARTOD flags for deployment "f"{deployment}")
        summary_results = qartod_summary_expanded(file_ds, parameters, deployment, test_name)
        statistics.update({f"{m}" : summary_results })

        # Add entry to summary statistics for full data record after last file
        if file == file_paths[-1]:
            # Open all expected data files and create merged full dataset
            merged_ds = [xr.open_dataset(single_file) for single_file in file_paths]
            merged_ds = merge_frames(merged_ds)
            deployment = "all"

            # Summary of flags from merged dataset
            print("Evaluating statistics on QARTOD flags for all deployments")
            merged_ds = parse_qartod_executed(merged_ds, parameters)
            summary_results = qartod_summary_expanded(merged_ds, parameters, deployment, test_name)
            statistics.update({ "all" : summary_results })

    # Create data frame from dictionary and check contents
    statistics = pd.DataFrame.from_dict(statistics, orient='index')
    statistics = statistics.set_index('deployment')
    return statistics

In [8]:
gross_range_stats = collect_statistics(expected_files, "gross_range")
gross_range_stats

Evaluating statistics on QARTOD flags for deployment 01
Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 03
Evaluating statistics on QARTOD flags for deployment 04
Evaluating statistics on QARTOD flags for all deployments


,sea_water_electrical_conductivity total,conductivity good,conductivity good %,conductivity suspect,conductivity suspect %,conductivity fail,conductivity fail %,sea_water_temperature total,temperature good,temperature good %,...,salinity suspect %,salinity fail,salinity fail %,sea_water_pressure total,pressure good,pressure good %,pressure suspect,pressure suspect %,pressure fail,pressure fail %
deployment,,,,,,,,,,,,,,,,,,,,,
01,8508,8508,100.0,0,0.0,0,0.0,8508,8508,100.0,...,0.00,0,0.0,8508,8392,98.64,116,1.36,0,0.0
02,2010802,2010802,100.0,0,0.0,0,0.0,2010802,2010802,100.0,...,0.09,0,0.0,2010802,1996702,99.30,14100,0.70,0,0.0
02,580942,580942,100.0,0,0.0,0,0.0,580942,580942,100.0,...,0.00,0,0.0,580942,575533,99.07,5409,0.93,0,0.0
03,69388,69388,100.0,0,0.0,0,0.0,69388,69388,100.0,...,4.21,0,0.0,69388,68237,98.34,1151,1.66,0,0.0
04,700020,700020,100.0,0,0.0,0,0.0,700020,700020,100.0,...,0.00,0,0.0,700020,693858,99.12,6162,0.88,0,0.0
all,3369660,3369660,100.0,0,0.0,0,0.0,3369660,3369660,100.0,...,0.14,0,0.0,3369660,3342722,99.20,26938,0.80,0,0.0


In [9]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
gross_range_stats.to_csv(folder_path+f"/gross_range-{refdes}-flag_statistics.csv", na_rep='NaN', mode='a')

#### Climatology test statistics

In [10]:
climatology_stats = collect_statistics(expected_files, "climatology")
climatology_stats

Evaluating statistics on QARTOD flags for deployment 01
Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 03
Evaluating statistics on QARTOD flags for deployment 04
Evaluating statistics on QARTOD flags for all deployments


,sea_water_electrical_conductivity total,sea_water_temperature total,temperature good,temperature good %,temperature suspect,temperature suspect %,temperature fail,temperature fail %,sea_water_practical_salinity total,salinity good,salinity good %,salinity suspect,salinity suspect %,salinity fail,salinity fail %,sea_water_pressure total
deployment,,,,,,,,,,,,,,,,
01,NaN,8508,8508,100.00,0,0.00,0,0.0,8508,7285,85.63,1223,14.37,0,0.0,NaN
02,NaN,2010802,2010802,100.00,0,0.00,0,0.0,2010802,1942780,96.62,68022,3.38,0,0.0,NaN
02,NaN,580942,580631,99.95,311,0.05,0,0.0,580942,580937,100.00,5,0.00,0,0.0,NaN
03,NaN,69388,66844,96.33,2544,3.67,0,0.0,69388,38875,56.03,30513,43.97,0,0.0,NaN
04,NaN,700020,699903,99.98,117,0.02,0,0.0,700020,693885,99.12,6135,0.88,0,0.0,NaN
all,NaN,3369660,3366688,99.91,2972,0.09,0,0.0,3369660,3263762,96.86,105898,3.14,0,0.0,NaN


In [11]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
climatology_stats.to_csv(folder_path+f"/climatology-{refdes}-flag_statistics.csv", na_rep='NaN', mode='a')